In [1]:
import pandas as pd
from functools import reduce

In [2]:
# Read CSVs
df1 = pd.read_csv('data/cap.csv')
df2 = pd.read_csv('data/dividends.csv')
df3 = pd.read_csv('data/stock_diff.csv')

# Merge DataFrames
merged_df = reduce(lambda left, right: pd.merge(left, right, on='Ticker'),
                    [df1, df2, df3])

years = [2021, 2022, 2023, 2024]
dfs = {}

for year in years:
    cols = [
        'Ticker', 'Name', 'Market Cap (HKD)', 'Sector',
        f'Revenue {year} (HKD)', f'EPS {year} (HKD)', f'First_Close_{year}',
        f'Last_Close_{year}', f'Close_Difference_{year}', f'Dividends_{year}',
          f'ROA {year} (%)'
    ]
    df_year = merged_df[cols].copy()
    df_year.columns = [
        'Ticker', 'Name', 'Market Cap (HKD)', 'Sector', 'Revenue (HKD)', 'EPS (HKD)',
        'First_Close', 'Last_Close', 'Close_Difference', 'Dividends', 'ROA (%)'
    ]
    # Convert to billions
    for col in ['Market Cap (HKD)', 'Revenue (HKD)']:
        df_year[col] = df_year[col] / 1_000_000_000
    df_year.rename(columns={
        'Market Cap (HKD)': 'Market Cap (B HKD)',
        'Revenue (HKD)': 'Revenue (B HKD)'
    }, inplace=True)
    # Calculate PE Ratio
    df_year['PE_Ratio'] = pd.to_numeric(
        df_year['Last_Close'], errors='coerce') / pd.to_numeric(
            df_year['EPS (HKD)'], errors='coerce')
    mask = (df_year['PE_Ratio'].isna()) | (
        df_year['PE_Ratio'] < 0) | (df_year['PE_Ratio'] > 1000)
    df_year.loc[mask, 'PE_Ratio'] = float('nan')
    # Calculate Growth
    df_year['Growth (%)'] = (((df_year['Close_Difference'] + df_year['Dividends']) /
                               df_year['First_Close']) * 100).round(2)
    df_year['Year'] = year
    # Rearrange columns
    desired_order = [
        'Ticker', 'Name', 'Year', 'Sector', 'Market Cap (B HKD)', 'Revenue (B HKD)',
        'EPS (HKD)', 'First_Close', 'Last_Close', 'Close_Difference', 'Dividends',
        'Growth (%)', 'ROA (%)', 'PE_Ratio'
    ]
    df_year = df_year[desired_order]
    dfs[str(year)] = df_year



In [ ]:
# Save all DataFrames to CSV (split version)
for year, df in dfs.items():
    #df['Year'] = int(year)
    df.to_csv(f'data/all_hk_stocks_{year}.csv', index=False)

In [ ]:
# # Combine all years into one DataFrame
all_years_df = pd.concat(dfs.values(), ignore_index=True)
all_years_df.to_csv('data/stocks_combined.csv', index=False)

In [16]:

# 1. With outliers: mean PE ratio per sector across all years
sector_pe_all = all_years_df.groupby('Sector')['PE_Ratio'].mean().round(2)

# 2. Without outliers: remove outliers per sector, then mean
sector_pe_no_outliers = {}
for sector in all_years_df['Sector'].unique():
    sector_data = all_years_df[all_years_df['Sector'] == sector]['PE_Ratio']
    Q1 = sector_data.quantile(0.05)
    Q3 = sector_data.quantile(0.95)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    filtered = sector_data[(sector_data >= lower) & (sector_data <= upper)]
    if not filtered.empty:
        sector_pe_no_outliers[sector] = filtered.mean().round(2)

# Merge both results into one DataFrame
result_df = pd.DataFrame({
    'Sector': sector_pe_all.index,
    'PE_Mean_All': sector_pe_all,
    'PE_Mean_95th': pd.Series(sector_pe_no_outliers)
})

result_df.to_csv('data/sector_pe_4y_mean.csv',index=False)

# Scoring - Paul

In [18]:
import numpy as np
PE_mean_df = pd.read_csv('data/sector_pe_4y_mean.csv')

In [21]:
PE_mean_df.columns

Index(['Sector', 'PE_Mean_All', 'PE_Mean_95th'], dtype='object')

In [20]:
all_years_df.columns

Index(['Ticker', 'Name', 'Year', 'Sector', 'Market Cap (B HKD)',
       'Revenue (B HKD)', 'EPS (HKD)', 'First_Close', 'Last_Close',
       'Close_Difference', 'Dividends', 'Growth (%)', 'ROA (%)', 'PE_Ratio'],
      dtype='object')

In [24]:
all_years_df = all_years_df.merge(PE_mean_df, on='Sector', how='left')
all_years_df

,Ticker,Name,Year,Sector,Market Cap (B HKD),Revenue (B HKD),EPS (HKD),First_Close,Last_Close,Close_Difference,Dividends,Growth (%),ROA (%),PE_Ratio,PE_Mean_All,PE_Mean_95th
0,0700.HK,TENCENT,2021,Communication Services,4625.990681,616.129800,25.956700,511.00119,408.93588,-102.06531,1.600000,-19.66,21.40,15.754540,33.10,22.87
1,1398.HK,ICBC,2021,Financial Services,2588.818538,1033.882300,1.045000,3.48622,3.37803,-0.10819,0.319911,6.07,16.84,3.232565,18.41,14.82
2,9988.HK,BABA-W,2021,Consumer Cyclical,2354.620137,789.017900,7.645000,226.83145,118.49850,-108.33295,0.000000,-47.76,14.19,15.500131,27.09,20.31
3,1288.HK,ABC,2021,Financial Services,2119.621542,791.456600,0.715000,1.98208,2.03792,0.05584,0.224415,14.14,17.45,2.850238,18.41,14.82
4,0941.HK,CHINA MOBILE,2021,Communication Services,1880.590254,933.083800,6.237000,32.63519,37.24722,4.61203,3.390000,24.52,20.77,5.971977,33.10,22.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9155,0362.HK,C ZENITH CHEM,2024,Utilities,0.000507,0.031562,-0.225700,0.10200,0.07700,-0.02500,0.000000,-24.51,-8.20,NaN,18.44,12.52
9156,0917.HK,QUNABOX GROUP,2024,Communication Services,NaN,1.473450,-9.240000,NaN,NaN,NaN,0.000000,NaN,-81.99,NaN,33.10,22.87
9157,1863.HK,"1863.HK,0P0000NB5Q,0",2024,NaN,NaN,9.997896,0.559416,1.37000,1.37000,0.00000,0.000000,0.00,6.56,2.448983,NaN,NaN
9158,2906.HK,WAH YAN-OLD,2024,NaN,NaN,0.054639,0.010000,0.38000,0.38000,0.00000,0.000000,0.00,603.41,38.000000,NaN,NaN


In [31]:
# all_years_df.drop(columns='PE_Mean_95th', axis=1, inplace=True)
# all_years_df

,Ticker,Name,Year,Sector,Market Cap (B HKD),Revenue (B HKD),EPS (HKD),First_Close,Last_Close,Close_Difference,Dividends,Growth (%),ROA (%),PE_Ratio,PE_Mean_All
0,0700.HK,TENCENT,2021,Communication Services,4625.990681,616.129800,25.956700,511.00119,408.93588,-102.06531,1.600000,-19.66,21.40,15.754540,33.10
1,1398.HK,ICBC,2021,Financial Services,2588.818538,1033.882300,1.045000,3.48622,3.37803,-0.10819,0.319911,6.07,16.84,3.232565,18.41
2,9988.HK,BABA-W,2021,Consumer Cyclical,2354.620137,789.017900,7.645000,226.83145,118.49850,-108.33295,0.000000,-47.76,14.19,15.500131,27.09
3,1288.HK,ABC,2021,Financial Services,2119.621542,791.456600,0.715000,1.98208,2.03792,0.05584,0.224415,14.14,17.45,2.850238,18.41
4,0941.HK,CHINA MOBILE,2021,Communication Services,1880.590254,933.083800,6.237000,32.63519,37.24722,4.61203,3.390000,24.52,20.77,5.971977,33.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9155,0362.HK,C ZENITH CHEM,2024,Utilities,0.000507,0.031562,-0.225700,0.10200,0.07700,-0.02500,0.000000,-24.51,-8.20,NaN,18.44
9156,0917.HK,QUNABOX GROUP,2024,Communication Services,NaN,1.473450,-9.240000,NaN,NaN,NaN,0.000000,NaN,-81.99,NaN,33.10
9157,1863.HK,"1863.HK,0P0000NB5Q,0",2024,NaN,NaN,9.997896,0.559416,1.37000,1.37000,0.00000,0.000000,0.00,6.56,2.448983,NaN
9158,2906.HK,WAH YAN-OLD,2024,NaN,NaN,0.054639,0.010000,0.38000,0.38000,0.00000,0.000000,0.00,603.41,38.000000,NaN


In [34]:
all_years_df['PE_top'] = all_years_df.PE_Mean_All *0.2 +all_years_df.PE_Mean_All
all_years_df['PE_bottom'] = all_years_df.PE_Mean_All -all_years_df.PE_Mean_All *0.2

Growth_weight = 0.5
Cap_weight = 0.2
ROA_weight = 0.2
PE_weight = 0.1

In [37]:
all_years_df['Growth Scoring'] = all_years_df['Growth (%)']*Growth_weight
all_years_df['Market Cap Scoring'] = (all_years_df['Market Cap (B HKD)']/100)*Cap_weight
all_years_df['ROA Scoring'] = all_years_df['ROA (%)']*ROA_weight
all_years_df['PE Scoring'] = np.where(all_years_df.PE_Ratio >all_years_df.PE_top, -20*PE_weight, 
                            np.where(all_years_df.PE_Ratio <all_years_df.PE_bottom, -20*PE_weight, 0))

In [38]:
all_years_df['Total'] = all_years_df['Growth Scoring']+all_years_df['Market Cap Scoring']+all_years_df['ROA Scoring']+all_years_df['PE Scoring']
all_years_df

,Ticker,Name,Year,Sector,Market Cap (B HKD),Revenue (B HKD),EPS (HKD),First_Close,Last_Close,Close_Difference,...,ROA (%),PE_Ratio,PE_Mean_All,PE_top,PE_bottom,Growth Scoring,Market Cap Scoring,ROA Scoring,PE Scoring,Total
0,0700.HK,TENCENT,2021,Communication Services,4625.990681,616.129800,25.956700,511.00119,408.93588,-102.06531,...,21.40,15.754540,33.10,39.720,26.480,-9.830,9.251981,4.280,-2.0,1.701981
1,1398.HK,ICBC,2021,Financial Services,2588.818538,1033.882300,1.045000,3.48622,3.37803,-0.10819,...,16.84,3.232565,18.41,22.092,14.728,3.035,5.177637,3.368,-2.0,9.580637
2,9988.HK,BABA-W,2021,Consumer Cyclical,2354.620137,789.017900,7.645000,226.83145,118.49850,-108.33295,...,14.19,15.500131,27.09,32.508,21.672,-23.880,4.709240,2.838,-2.0,-18.332760
3,1288.HK,ABC,2021,Financial Services,2119.621542,791.456600,0.715000,1.98208,2.03792,0.05584,...,17.45,2.850238,18.41,22.092,14.728,7.070,4.239243,3.490,-2.0,12.799243
4,0941.HK,CHINA MOBILE,2021,Communication Services,1880.590254,933.083800,6.237000,32.63519,37.24722,4.61203,...,20.77,5.971977,33.10,39.720,26.480,12.260,3.761181,4.154,-2.0,18.175181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9155,0362.HK,C ZENITH CHEM,2024,Utilities,0.000507,0.031562,-0.225700,0.10200,0.07700,-0.02500,...,-8.20,NaN,18.44,22.128,14.752,-12.255,0.000001,-1.640,0.0,-13.894999
9156,0917.HK,QUNABOX GROUP,2024,Communication Services,NaN,1.473450,-9.240000,NaN,NaN,NaN,...,-81.99,NaN,33.10,39.720,26.480,NaN,NaN,-16.398,0.0,NaN
9157,1863.HK,"1863.HK,0P0000NB5Q,0",2024,NaN,NaN,9.997896,0.559416,1.37000,1.37000,0.00000,...,6.56,2.448983,NaN,NaN,NaN,0.000,NaN,1.312,0.0,NaN
9158,2906.HK,WAH YAN-OLD,2024,NaN,NaN,0.054639,0.010000,0.38000,0.38000,0.00000,...,603.41,38.000000,NaN,NaN,NaN,0.000,NaN,120.682,0.0,NaN
